In [ ]:
#Team ABIDE_fmri with The BrainHack School Montreal
#Collaborators: Emily Chen, Ziad Guenoun, Andréanne Proulx, Mikkel Schöttner
#Date: May 2020

In [1]:
#Import modules
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nilearn.datasets

%matplotlib inline

In [ ]:
#Fetch the ABIDE dataset using nilearn
#nilearn.datasets.fetch_abide_pcp(data_dir=None, n_subjects=None, pipeline='cpac', band_pass_filtering=False, global_signal_regression=False, derivatives=['func_preproc'], quality_checked=True, url=None, verbose=1)

In [11]:
#Assign the data 
pheno_file = ('/Users/emilychen/GitHub/nilearn_data/ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv')
pheno = pd.read_csv(pheno_file,index_col=1)
pheno.head() #Print beginning of file

,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
Unnamed: 0.1,,,,,,,,,,,,,,,,,,,,,
1,0,50002,1,50002,PITT,no_filename,1,1,16.77,1,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
2,1,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,1,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,2,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,1,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
4,3,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,2,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
5,4,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,1,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


### Sites ID Legend (alphabetized)
* **CALTECH**: California Institute of Technology
* **CMU**: Carnegie Mellon University
* **KKI**: Kennedy Krieger Institute 
* **MAX_MUN**: Ludwig Maximilians University Munich
* **NYU**: NYU Langone Medical Center
* **OLIN**: Olin, Institute of Living at Hartford Hospital
* **OHSU**: Oregon Health and Science University
* **SDSU**: San Diego State University 
* **SBL**: Social Brain Lab at the BCN NIC UMC Groningen and Netherlands Institute for Neurosciences
* **STANFORD**: Stanford School of Medicine
* **TRINITY**: Trinity Centre for Health Sciences
* **UCLA_1**: University of California, Los Angeles Sample 1
* **UCLA_2**: University of California, Los Angeles Sample 2
* **LEUVEN_1**: University of Leuven Sample 1
* **LEUVEN_2**: University of Leuven Sample 2
* **UM_1**: University of Michigan Sample 1
* **UM_2**: University of Michigan Sample 2
* **PITT**: University of Pittsburgh School of Medicine
* **USM**: University of Utah School of Medicine
* **YALE**: Yale Child Study Center at the Yale School of Medicine

In [5]:
#Print the unique SITE_ID values in the .csv file
pheno["SITE_ID"].unique()

array(['PITT', 'OLIN', 'OHSU', 'SDSU', 'TRINITY', 'UM_1', 'UM_2', 'USM',
       'YALE', 'CMU', 'LEUVEN_1', 'LEUVEN_2', 'KKI', 'NYU', 'STANFORD',
       'UCLA_1', 'UCLA_2', 'MAX_MUN', 'CALTECH', 'SBL'], dtype=object)

[use this scikitlearn resource](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) and [this](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html#sklearn.model_selection.GroupKFold). Also use [this paper](https://hal.inria.fr/hal-01545002/file/paper.pdf).

* site based CV using scikit learn, getting 10 splits

In [10]:
#Create dataframes for each site to calculate summary statistics per site
pitt = pheno[pheno["SITE_ID"]=="PITT"]
pitt.head()

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [12]:
#Summary statistics and distribution of AGE_AT_SCAN for PITT site
print(pitt["AGE_AT_SCAN"].describe())

count    57.000000
mean     18.903860
std       6.878105
min       9.330000
25%      13.590000
50%      16.990000
75%      22.640000
max      35.200000
Name: AGE_AT_SCAN, dtype: float64


In [ ]:
#Summary statistics and distribution of AGE_AT_SCAN
print(pheno["AGE_AT_SCAN"].describe())

In [ ]:
#Create a simple histogram of these ages using seaborn
sns.distplot(pheno["AGE_AT_SCAN"],
             bins=100,
             color="green",
             rug=True,
            )

In [ ]:
#Log normalization for AGE_AT_SCAN
log_age = np.log(pheno["AGE_AT_SCAN"])
sns.distplot(log_age,
            bins=100,
            color="red",
            rug=True,
            )

In [ ]:
#Countplot of HANDEDNESS_CATEGORY
sns.countplot(pheno["HANDEDNESS_CATEGORY"])

In [ ]:
#Countplot of diagnostic group (DX_GROUP)
#1=autism, 2=control
sns.countplot(pheno["DX_GROUP"])

In [ ]:
#Bivariate visualization: linear x linear
new_pheno = pheno.replace(-9999,np.nan) #Remove instances of -9999 in data
sns.regplot(x=new_pheno["AGE_AT_SCAN"], y=new_pheno["FIQ"])
plt.xlabel("Age at Scan")
plt.ylabel("FIQ")